In [1]:
print("hello world")

hello world


In [2]:
pip install -U sentence-transformers

  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none

In [3]:
pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 5.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.70.0
    Uninstalling grpcio-1.70.0:
      Successfully uninstalled grpcio-1.70.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/home/tanim/miniconda3/envs/wafi_ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
documents = [
    { "name": "The Time Machine", "description": "A man travels through time and witnesses the evolution of human", "year": 1895 },
    { "name": "Ender's Game", "description": "A young boy is trained to become a military leader in a war against", "year": 1985 },
    { "name": "Brave New World", "description": "A dystopian society where people are genetically engineered and", "year": 1932 },
    { "name": "The Hitchhiker's Guide to the Galaxy", "description": "A comedic science fiction series following", "year": 1979 },
    { "name": "Dune", "description": "A desert planet is the site of political intrigue and power struggles.", "year": 1965 },
    { "name": "Foundation", "description": "A mathematician develops a science to predict the future of humanity", "year": 1951 },
    { "name": "Snow Crash", "description": "A futuristic world where the internet has evolved into a virtual real", "year": 1992 },
    { "name": "Neuromancer", "description": "A hacker is hired to pull off a near-impossible hack and gets pulled", "year": 1984 },
    { "name": "The War of the Worlds", "description": "A Martian invasion of Earth throws humanity into chaos.", "year": 1898 },
    { "name": "The Hunger Games", "description": "A dystopian society where teenagers are forced to fight to the", "year": 2008 },
    { "name": "The Andromeda Strain", "description": "A deadly virus from outer space threatens to wipe out human", "year": 1969 },
    { "name": "The Left Hand of Darkness", "description": "A human ambassador is sent to a planet where the inhab", "year": 1969 },
    { "name": "The Three-Body Problem", "description": "Humans encounter an alien civilization that lives in a dy", "year": 2008 }
]


In [8]:
qdrant = QdrantClient(":memory:")

In [9]:
qdrant.recreate_collection(
    collection_name="books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_59658/3078561208.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [37]:
qdrant.upload_records(
    collection_name="books",
    records=[
        models.Record(
            id=i,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc
        )
        for i, doc in enumerate(documents)
    ],
)

/tmp/ipykernel_59658/3591890698.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [48]:
hits = qdrant.search(
    collection_name="books",
    query_vector=encoder.encode("Science fiction").tolist(),
    limit=5,    
)

print("Result:")
for hit in hits:
    print(hit.payload, "score:", hit.score)

Result:
{'name': "The Hitchhiker's Guide to the Galaxy", 'description': 'A comedic science fiction series following'} score: 0.5062118614987268
{'name': 'Foundation', 'description': 'A mathematician develops a science to predict the future of humanity'} score: 0.41323166162642166
{'name': 'Brave New World', 'description': 'A dystopian society where people are genetically engineered and'} score: 0.4003578907210523
{'name': 'The War of the Worlds', 'description': 'A Martian invasion of Earth throws humanity into chaos.'} score: 0.3413500260132142
{'name': 'The Hunger Games', 'description': 'A dystopian society where teenagers are forced to fight to the'} score: 0.32862648252101495


/tmp/ipykernel_59658/3076539486.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [47]:
hits = qdrant.search(
    collection_name="books",
    query_vector=encoder.encode("Science fiction").tolist(),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="year",
                range=models.Range(
                    gte=2000,
                )
            )
        ]
    ),
)

print("Result:", hits)
for hit in hits:
    print(hit.payload, "score:", hit.score)

Result: []


/tmp/ipykernel_59658/2010950786.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(
